In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.contrib.slim as slim
import sys
import os
import cv2
from tensorflow.python.client import timeline
from datetime import datetime
import network as vgg

def get_init_restorer():
    variables_to_restore = []
    # for var in slim.get_model_variables():
    for var in tf.global_variables():
        variables_to_restore.append(var)
    
    return tf.train.Saver(variables_to_restore)

scale_size = vgg.vgg_16.default_image_size
cache_folder = '/export/home/qliu24/qing_voting_139/qing_voting_py/cache/'
checkpoints_dir = os.path.join(cache_folder, 'checkpoints_vgg')
tf.logging.set_verbosity(tf.logging.INFO)
with tf.device('/cpu:0'):
    input_images = tf.placeholder(tf.float32, [1, None, None, 3])

vgg_var_scope = 'vgg_16'
with tf.variable_scope(vgg_var_scope, reuse=False):
    with slim.arg_scope(vgg.vgg_arg_scope()):
        _, end_points = vgg.vgg_16(input_images)
        
restorer = get_init_restorer()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
init_op = tf.global_variables_initializer()
# Run the session:
sess = tf.Session(config=config)
# init
print(str(datetime.now()) + ': Start Init')

restorer.restore(sess, os.path.join(checkpoints_dir, 'vgg_16.ckpt'))

print(str(datetime.now()) + ': Finish Init')

2017-07-26 16:47:12.219586: Start Init
INFO:tensorflow:Restoring parameters from /export/home/qliu24/qing_voting_139/qing_voting_py/cache/checkpoints_vgg/vgg_16.ckpt
2017-07-26 16:47:13.706342: Finish Init


In [3]:


end_points['vgg_16/fc8/reduced']

<tf.Tensor 'vgg_16/fc8/reduced:0' shape=(1, 1000) dtype=float32>

In [ ]:
import math
import pickle
from scipy.spatial.distance import cdist
import scipy.io as sio
from myresize import myresize

scale_size = 224
featDim = 512
img_mean = np.array([104., 117., 124.]).reshape(1,1,3)

dir_mat = '/export/home/qliu24/VC_adv_data/cihang/adv_cls_patches/adv_mat_file/'
file_list = '/export/home/qliu24/VC_adv_data/cihang/adv_cls_patches/car_test.txt'
with open(file_list, 'r') as fh:
    content = fh.readlines()
    
img_list = [x.strip().split() for x in content]
img_num = len(img_list)
print('total number of images: {0}'.format(img_num))

img_ori=[]
img_fool=[]
for nn in range(10):
    file_mat = os.path.join(dir_mat, '{0}.mat'.format(img_list[nn][0]))
    assert(os.path.isfile(file_mat))
    matcontent = sio.loadmat(file_mat)
    im_ori = matcontent['im_ori'][:,:,[2,1,0]].astype('float64')
    im_ori -= img_mean
    im_ori= 
    im_fool = matcontent['im_fool'][:,:,[2,1,0]]